# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [ ]:
data = pd.read_csv('/datasets/data.csv')

Выведем первые 20 строк таблицы, также воспользуемся методами .info() и .describe() для полного ознакомления с данными:

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Наблюдения (страности в данных) на основе приведенной информации: 
* В строке ***education*** значения в разном регистре  
* В строке ***dob_years*** минимальное значение возраста равно 0 
* Значения -1 и 20 в строке ***children***
* Столбец ***days_employed*** явно с ошибкой, причем с несколькими. Начиная с среднего значения в 172 года заканчивая -50 минимальными годами и 1101 максимальным
* В строках ***days_employed*** и ***total_income*** тип данных *float*, я бы перевел в *int*, для удобства восприятия 
* Отсутствуют данные в столбцах ***days_employed*** и ***total_income***, судя по тому, кол-во заполненных строк одинаковое, стоит предположить, что данные пропущены в одних и тех же строках таблицы, стоит это проверить
* Остальные столбцы выглядят нормально

Заметки: 
* неудобно как по мне опыт в столбце ***days_employed*** считать в днях, предлагаю вывести опыт работы в годах  
* Так же неудобно название ***total_income*** , заменить на ***month_income***, для удобства и поделить на 1000, получив значение зарплаты в тысячах

In [ ]:
data = data.rename(columns={'total_income': 'month_income', 'days_employed': 'years_employed'})

In [ ]:
data['years_employed'] = data['years_employed'] / 365
data['month_income'] = data['month_income'] / 1000

Выведем датафрейм для того, чтообы проверить внесенные изменения:

In [ ]:
data

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,month_income,purpose
0,1,-23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253.875639,покупка жилья
1,1,-11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112.080014,приобретение автомобиля
2,0,-15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145.885952,покупка жилья
3,3,-11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267.628550,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158.616078,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-12.409087,43,среднее,1,гражданский брак,1,F,компаньон,0,224.791862,операции с жильем
21521,0,942.294258,67,среднее,1,женат / замужем,0,F,пенсионер,0,155.999807,сделка с автомобилем
21522,1,-5.789991,38,среднее,1,гражданский брак,1,M,сотрудник,1,89.672561,недвижимость
21523,3,-8.527347,38,среднее,1,женат / замужем,0,M,сотрудник,1,244.093051,на покупку своего автомобиля


**Вывод**

В выводах отражены наблюдения по каждому из столбцов таблицы:
* ***children***: Минимальные значения (если их несколько), равные -1, следует исправить на 1, скорее всего человеческий фактор, просто ошиблись при вводе. Также следует выяснить причину возникновения максимального значения в 20 детей (возможно не является ошибкой, если случай единичный).
* ***dob_years***: Минимальное значение возраста заемщика равно 0, следует исправить на среднее значение возраста заемщика (более детально подходить к замене 0 не вижу смысла, т.к. возраст не является релевантным признаком возврата кредита для заказчика в данной задаче). Также можно просто удалить по сути ненужную нам строку.
* ***days_employed (years_employed)***: Название стобца было изменено, как и данные в нем, для удобства понимания данных и дальнейшей работы с ними. В данном столбце также присутствуют ошибки, предлагаю удалить и эту строку.
* ***education***: Следует привести значения к нижнему регистру.
* В строках ***education_id, family_status, family_status_id, gender, income_type, debt***: недочетов на данном этапе работы не обнаружено.
* ***total_income (month_income)***: Название стобца было изменено, как и данные в нем, для удобства понимания данных и дальнейшей работы с ними. Также следует заменить тип данных в данном столбце.
* ***purpose***: В дальнейшем требуется лемматизация для дальнейшей категоризации данных.

## Шаг 2. Предобработка данных

### Обработка странных значений

На этапе ознакомления с информацией были выявлены некоторые ошибки и недочеты в данных, пришло время их исправить.   
Начнем с некорректных данных о количестве детей:

Воспользуемся методом .value_counts(), для того, чтобы узнать количество строк таблицы для каждого уникального значения кол-ва детей:

In [ ]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Из полученных данных можно сделать вывод о том, что 20 детей, это ошибка, скорее всего вызванная человеческим фактором (слишком много заемщиков с 20 детьми).  
Также можно вспомнить о том, что для нашей задачи существенным признаком является наличие ребенка, а не их количество, следовательно можно не менять значения равные 20, а вот значения равные -1 поменять на 1 нужно, для дальнейшей категоризации.

In [ ]:
data = data.loc[data['children'] <= 5]
data = data.loc[data['children'] >= 0]

Проверка полученных данных:

In [ ]:
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Также были замечены ошибки в столбце ***years_employed***:

In [ ]:
data['years_employed'].describe()

count    19240.000000
mean       173.040605
std        386.106694
min        -50.380685
25%         -7.526673
50%         -3.298450
75%         -0.793809
max       1100.699727
Name: years_employed, dtype: float64

**Вывод**

В таблице были исправлены все странные значения и были удалены столбцы, не относящиеся к поставленной задаче.

Также были замечены ошибки в столбце ***years_employed*** и в столбце ***dob_years***, но данные в них не помогут нам ответить на вопросы поставленные заказчиком, так что оставим данные без исправлений.  
Если бы эти данные были важны, следовало бы решить проблемы с ними, а именно:
* Найти причину отрицательных значений в столбце ***years_employed*** и скорее всего взять значения по модулю
* Найти причину и исправить значения равные 0 в столбце ***dob_years***

### Обработка пропусков

Заменим отсутствующие данные о зарплате заемщика медианными значениями зарплаты по профессии заемщика.

Для начала ознакомимся с данными:

In [ ]:
data['month_income'].describe()

count    19240.000000
mean       167.448790
std        103.105123
min         20.667264
25%        103.000126
50%        145.020801
75%        203.444650
max       2265.604029
Name: month_income, dtype: float64

In [ ]:
data.groupby('income_type')['month_income'].agg(['median', 'mean'])

,median,mean
income_type,,
безработный,131.339752,131.339752
в декрете,53.829131,53.829131
госслужащий,150.420150,170.743143
компаньон,172.517419,202.585824
пенсионер,118.480837,137.198468
предприниматель,499.163145,499.163145
сотрудник,142.587589,161.365012
студент,98.201625,98.201625


Была написана функция, для заполнения пропусков в данных медианным значением зарплаты по профессии заемщика:

In [ ]:
def fill_missings_month_income(work_names):
    for index in range(len(work_names)):
        month_income_mean = data[data['income_type'] == work_names[index]]['month_income'].median()
        data[data['income_type'] == work_names[index]] = data[data['income_type'] == work_names[index]].fillna(month_income_mean)
    return data

In [ ]:
income_types = data['income_type'].unique()
fill_missings_month_income(income_types)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,month_income,purpose
0,1,-23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253.875639,покупка жилья
1,1,-11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112.080014,приобретение автомобиля
2,0,-15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145.885952,покупка жилья
3,3,-11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267.628550,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158.616078,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-12.409087,43,среднее,1,гражданский брак,1,F,компаньон,0,224.791862,операции с жильем
21521,0,942.294258,67,среднее,1,женат / замужем,0,F,пенсионер,0,155.999807,сделка с автомобилем
21522,1,-5.789991,38,среднее,1,гражданский брак,1,M,сотрудник,1,89.672561,недвижимость
21523,3,-8.527347,38,среднее,1,женат / замужем,0,M,сотрудник,1,244.093051,на покупку своего автомобиля


Проверка данных и ознакомление с новыми значениями зарплат по профессиям:

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   years_employed    21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  month_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.6+ MB


In [ ]:
 data.groupby('income_type')['month_income'].agg(['median', 'mean'])

,median,mean
income_type,,
безработный,131.339752,131.339752
в декрете,53.829131,53.829131
госслужащий,150.420150,168.701053
компаньон,172.517419,199.587312
пенсионер,118.480837,135.199451
предприниматель,499.163145,499.163145
сотрудник,142.587589,159.494067
студент,98.201625,98.201625


**Вывод**

В таблице были заменены все пропуски в значения зарплат заемщиков на медианные значения зарплат по профессии заемщика.

### Замена типа данных

При онакомлении с данными был сделан вывод о том, что следует заменить тип данных с строке ***month_income*** и ***years_employed*** с *float* на *int*:

In [ ]:
data['month_income'] = data['month_income'].astype('int')
data['years_employed'] = data['years_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   years_employed    21402 non-null  int64 
 2   dob_years         21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  month_income      21402 non-null  int64 
 11  purpose           21402 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.6+ MB


Проверка данных:

In [ ]:
data

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,month_income,purpose
0,1,-23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья
1,1,-11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля
2,0,-15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья
3,3,-11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-12,43,среднее,1,гражданский брак,1,F,компаньон,0,224,операции с жильем
21521,0,942,67,среднее,1,женат / замужем,0,F,пенсионер,0,155,сделка с автомобилем
21522,1,-5,38,среднее,1,гражданский брак,1,M,сотрудник,1,89,недвижимость
21523,3,-8,38,среднее,1,женат / замужем,0,M,сотрудник,1,244,на покупку своего автомобиля


**Вывод**

Был заменен тип данных в строке ***month_income*** и ***years_employed***.

### Обработка дубликатов

Найдем и удалим все дубликаты в таблице:

Исправим значения в столбце ***education***

In [ ]:
data['education'] = data['education'].str.lower()

Проверим данные:

In [ ]:
data

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,month_income,purpose
0,1,-23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья
1,1,-11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля
2,0,-15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья
3,3,-11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-12,43,среднее,1,гражданский брак,1,F,компаньон,0,224,операции с жильем
21521,0,942,67,среднее,1,женат / замужем,0,F,пенсионер,0,155,сделка с автомобилем
21522,1,-5,38,среднее,1,гражданский брак,1,M,сотрудник,1,89,недвижимость
21523,3,-8,38,среднее,1,женат / замужем,0,M,сотрудник,1,244,на покупку своего автомобиля


Для начала найдем количество дубликатов сочетанием методов .duplicated() и .sum():

In [ ]:
data.duplicated().sum()

72

Удалим дубликаты методом drop_duplicates():

In [ ]:
data = data.drop_duplicates().reset_index(drop = True)

Проверка данных:

In [ ]:
data.duplicated().sum()

0

**Вывод**

В данных были удалены все дубликаты.

### Лемматизация

Для дальнейшей категоризации данных по цели кредита, необходимо лемматизировать столбец ***'purpose'***:

In [ ]:
lemmas_list = []
for purpose in data['purpose']:
    lemmas_list.extend(m.lemmatize(purpose))

In [ ]:
print(Counter(lemmas_list)) 

Counter({' ': 33375, '\n': 21330, 'недвижимость': 6314, 'покупка': 5864, 'жилье': 4437, 'автомобиль': 4278, 'образование': 3988, 'с': 2900, 'операция': 2587, 'свадьба': 2313, 'свой': 2218, 'на': 2206, 'строительство': 1867, 'высокий': 1367, 'получение': 1309, 'коммерческий': 1303, 'для': 1286, 'жилой': 1222, 'сделка': 936, 'заниматься': 904, 'дополнительный': 899, 'проведение': 763, 'сыграть': 760, 'сдача': 649, 'семья': 637, 'собственный': 628, 'со': 623, 'ремонт': 604, 'подержанный': 481, 'подержать': 471, 'приобретение': 459, 'профильный': 432})


**Вывод**

При лемматизации столбца ***'purpose'*** были выявлены основные цели взятия кредита:
* недвижимость
* авто
* образование
* свадьба  
  
В дальнейшем по ним данные будут категоризированы.

### Категоризация данных

Чтобы категоризировать данные по цели кредита, предлагаю создать дополнительный столбец с категорией причины.  
Для этого была создана функция *purpose_id*:

In [ ]:
def purpose_id(purpose):
    lemma = m.lemmatize(purpose)
    if 'свадьба' in lemma:
        return 'свадьба'
    elif 'автомобиль' in lemma:
        return 'авто'
    elif 'недвижимость' in lemma or 'жилье' in lemma:
        return 'недвижимость'
    elif 'образование' in lemma:
        return 'образование'
    else:
        return -1

data['purpose_id'] = data['purpose'].apply(purpose_id)
data['purpose_id'].value_counts()

недвижимость    10751
авто             4278
образование      3988
свадьба          2313
Name: purpose_id, dtype: int64

Далее делаем категоризацию данных по уровню достатка:

Госстатистика разделяет уровень жизни россиян по доходам на такие категории:  
* Крайняя нищета – доходы ниже 7-8 тыс. в месяц  
* Нищета – доходы от 8 до 12 тысяч рублей в месяц  
* Бедность – доходы от 12 до 20 тысяч рублей в месяц  
* Выше бедности – доходы от 20 до 30 тысяч рублей в месяц  
* Средний достаток — доходы от 30 до 60 тысяч рублей в мес
* Богатые – доходы свыше 90 тысяч рублей в месяц  
* Сверхбогатые – доходы свыше 150 тысяч в месяц

На основе этого, изменив градацию категорий, будем категоризировать наши данные.  
Так как минимальная зарплата заемщиков равна 20 т.р., то отбросим категории крайняя нищета, нищета и бедность.  
Изменим градацию категорий:
* Низкий достаток – доходы от 20 до 40 тысяч рублей в месяц  
* Средний достаток — доходы от 40 до 70 тысяч рублей в мес
* Достаток выше среднего – доходы от 70 до 100
* Высокий достаток – доходы от 100 до 300 тысяч рублей в месяц  
* Сверхбогатые – доходы свыше 300 тысяч в месяц

Чтобы категоризировать столбец ***month_income***, предлагаю создать новый столбец с категорией достатка.  
Для этого была создана функция *income_id*:

In [ ]:
def income_id(month_income):
    if 20 <= month_income <= 40:
        return 'низкий'
    elif 40 < month_income <= 70:
        return 'средний'
    elif 70 < month_income <= 100:
        return 'выше среднего'
    elif 100 < month_income <= 300:
        return 'высокий'
    elif month_income > 300:
        return 'сверхбогатый'
    else:
        return -1

data['month_income_id'] = data['month_income'].apply(income_id)
data['month_income_id'].value_counts()

высокий          15306
выше среднего     3026
сверхбогатый      1457
средний           1404
низкий             137
Name: month_income_id, dtype: int64

Далее следует категоризировать данные по количеству детей.  
Для заказчика важен фактор наличия или отсутвия детей, так что предлагаю создать новый столбец и сделать 2 категории данных по количеству детей: 
* нет детей
* есть дети

In [ ]:
def children_id(children):
    if children == 0:
        return 'нет детей'
    else:
        return 'есть дети'
    
data['children_id'] = data['children'].apply(children_id)
data['children_id'].value_counts()

нет детей    14091
есть дети     7239
Name: children_id, dtype: int64

Полученная после всех преобразований таблица:

In [ ]:
data

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,month_income,purpose,purpose_id,month_income_id,children_id
0,1,-23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья,недвижимость,высокий,есть дети
1,1,-11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля,авто,высокий,есть дети
2,0,-15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья,недвижимость,высокий,нет детей
3,3,-11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование,образование,высокий,есть дети
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу,свадьба,высокий,нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21325,1,-12,43,среднее,1,гражданский брак,1,F,компаньон,0,224,операции с жильем,недвижимость,высокий,есть дети
21326,0,942,67,среднее,1,женат / замужем,0,F,пенсионер,0,155,сделка с автомобилем,авто,высокий,нет детей
21327,1,-5,38,среднее,1,гражданский брак,1,M,сотрудник,1,89,недвижимость,недвижимость,выше среднего,есть дети
21328,3,-8,38,среднее,1,женат / замужем,0,M,сотрудник,1,244,на покупку своего автомобиля,авто,высокий,есть дети


**Вывод**

Была выполнена категоризация данных для ответа на вопросы, поставленные заказчиком.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data.groupby('children_id')['debt'].agg(['count','mean']).sort_values(by = 'mean')

,count,mean
children_id,,
нет детей,14091,0.075438
есть дети,7239,0.092416


**Вывод**

Заемщики, не имеющие детей, менее склонны к задолженности по возврату кредита. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data.groupby('family_status')['debt'].agg(['count','mean']).sort_values(by = 'mean')

,count,mean
family_status,,
вдовец / вдова,951,0.066246
в разводе,1189,0.070648
женат / замужем,12260,0.075612
гражданский брак,4134,0.093130
Не женат / не замужем,2796,0.097639


**Вывод**

Вдовцы и вдовы наименее склонны к задолженности по возврату кредита. Люди никогда не вступавшие в брак, чаще имеют задолженности по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data.groupby('month_income_id')['debt'].agg(['count','mean']).sort_values(by = 'mean')

,count,mean
month_income_id,,
средний,1404,0.069801
сверхбогатый,1457,0.072066
низкий,137,0.072993
высокий,15306,0.082713
выше среднего,3026,0.083609


**Вывод**

Сверхбогатые люди наименее склонны к задолженности по возврату кредита. Люди в высоким доходом и доходом выше сдеднего чаще имеют задолженность.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data.groupby('purpose_id')['debt'].agg(['count','mean']).sort_values(by = 'mean')

,count,mean
purpose_id,,
недвижимость,10751,0.072551
свадьба,2313,0.079118
образование,3988,0.092528
авто,4278,0.093502


**Вывод**

Люди, берущие кредит на недвижимость наименее склонны к задолженности по возврату кредита. Заемщик, который берет кредит на авто, наиболее склонен к просрочке платежа по кредиту.

## Шаг 4. Общий вывод

В ходе работы над данным проектом были получены ответы на вопросы, поставленные заказчиком, а именно:

- Есть ли зависимость между наличием детей и возвратом кредита в срок? --- Да, есть: наиболее ответственные заемщики те, у которых нет детей.
- Есть ли зависимость между семейным положением и возвратом кредита в срок? --- Да, есть: наиболее ответственными заемщиками являются вдовы и вдовцы.
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок? --- Да, есть: наиболее ответственны сверхбогатые заемщики. 
- Как разные цели кредита влияют на его возврат в срок? --- Люди, берущие кредит на недвижимость наименее склонны к задолженности по возврату кредита.

По исследованным данным можно составить портрет заемщика, у которого, вероятнее всего не будет проблем с задолженностью по кредиту:

- Не имеет детей
- Находится или был в браке
- Очень богатый человек или человек со средним достатком
- Цель кредита - недвижимость